In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as stats
import operator
import math
import warnings
import openpyxl
import random
warnings.filterwarnings('ignore')
from sklearn import tree
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, accuracy_score
import tensorflow as tf
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import ClusterCentroids
from imblearn.under_sampling import CondensedNearestNeighbour
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
import pandas as pd

# 한글 글꼴체 변경
plt.rcParams['font.family'] ='Malgun Gothic'
# df.head() 이런거 했을 때, 컬럼이 생략되지 않고, 모든 컬럼 뜨게
pd.set_option('display.max_columns', None)
# 지수 표기법 대신에 소수점으로 표시하는코드
pd.options.display.float_format = '{:.5f}'.format
# 값 길이 제한 X
pd.set_option('display.max_colwidth', None) # 값 길이 제한 없음

### ✅ Customer Segmentation

**고객 해지 위험 예측 서비스 사이트에 사용하기 위해** 고객 위험도 분류를 시행

모델이 예측한 해지율을 기반으로 고객을 "양호", "보통", "위험" 3가지 그룹으로 세분화

🤔 어떻게 세분화?

#### 모델이 예측한 해지율을 뽑아보자.

In [4]:
import joblib
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder

model_path = 'data/file_pkl/lightgbm_model.pkl'
loaded_model = joblib.load(model_path)
# 스케일러 불러오기
robust_scaler = joblib.load('data/file_pkl/robust_scaler.pkl')
minmax_scaler = joblib.load('data/file_pkl/minmax_scaler.pkl')

# ----------------------------------------------------------------------------------------------

# 데이터 불러오기
df_test = pd.read_csv("data/full_data/TPS_cancel_data_Final.csv")
df = df_test

df['INHOME_RATE'] = df['INHOME_RATE'].astype(str)

print(f"df INHOME_RATE dtype: {df['INHOME_RATE'].dtype}\n")

# ----------------------------------------------------------------------------------------------

df_modeling = df.drop(columns=['sha2_hash','p_mt','churn'])

# 레이블 인코딩 수행
label_encoders = {}
# object 타입 
for column in df_modeling.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df_modeling[column] = le.fit_transform(df_modeling[column])
    label_encoders[column] = le 

for col, le in label_encoders.items():
    print(f"🔹 {col} 레이블 인코딩 매핑:")
    label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))  # 원본값 → 인코딩된 값
    print(label_mapping)
    print("-" * 50)

# ✅ 불러온 스케일러 적용
robust_columns = ['TOTAL_USED_DAYS', 'CH_HH_AVG_MONTH1', 'MONTHS_REMAINING']  # RobustScaler 적용 대상
minmax_columns = [col for col in df_modeling.columns if col not in robust_columns + ['churn']]  # 나머지는 MinMaxScaler 적용

# ✅ test 데이터에 스케일링 적용
df_modeling[robust_columns] = robust_scaler.transform(df_modeling[robust_columns])
df_modeling[minmax_columns] = minmax_scaler.transform(df_modeling[minmax_columns])

probabilities = loaded_model.predict_proba(df_modeling)  # 각 클래스에 대한 확률 반환
predictions = (probabilities[:, 1] >= 0.5).astype(int)  # 기본 Threshold = 0.5 사용, 기본으로 Threshold는 0.5로 적용됩니다.

# 원래 데이터에 예측 결과 추가
df['probability_0'] = probabilities[:, 0]  # 해지하지 않을 확률
df['probability_1'] = probabilities[:, 1]  # 해지할 확률 -> 이거 위주로 보셔야합니다
df['prediction'] = predictions             # 예측 결과 (0: 미해지, 1: 해지)

print("완전히 끝!!✔")

df INHOME_RATE dtype: object

🔹 AGE_GRP10 레이블 인코딩 매핑:
{'10대': 0, '20대': 1, '30대': 2, '40대': 3, '50대': 4, '60대': 5, '70대': 6, '80대': 7, '90대이상': 8, '연령없음': 9}
--------------------------------------------------
🔹 AGMT_END_SEG 레이블 인코딩 매핑:
{'약정만료 1개월': 0, '약정만료전 12개월이상': 1, '약정만료전 1~2개월': 2, '약정만료전 1개월': 3, '약정만료전 2~3개월': 4, '약정만료전 3~6개월': 5, '약정만료전 6~9개월': 6, '약정만료전 9~12개월': 7, '약정만료후 12개월이상': 8, '약정만료후 1개월~2개월': 9, '약정만료후 2개월~3개월': 10, '약정만료후 3~6개월': 11, '약정만료후 6~9개월': 12, '약정만료후 9~12개월': 13}
--------------------------------------------------
🔹 BUNDLE_YN 레이블 인코딩 매핑:
{'N': 0, 'Y': 1}
--------------------------------------------------
🔹 CH_LAST_DAYS_BF_GRP 레이블 인코딩 매핑:
{'2주일전': 0, '3개월내없음': 1, '3주일전': 2, '4주일전': 3, '일주일내': 4, '일주일전': 5}
--------------------------------------------------
🔹 CONTENT_USE_YN 레이블 인코딩 매핑:
{'N': 0, 'Y': 1}
--------------------------------------------------
🔹 INHOME_RATE 레이블 인코딩 매핑:
{'0.0': 0, '10.0': 1, '100.0': 2, '20.0': 3, '30.0': 4, '40.0': 5, '50.0': 6, '60.0'

In [7]:
df[['sha2_hash','probability_0','probability_1','prediction','churn']]

,sha2_hash,probability_0,probability_1,prediction,churn
0,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,0.26084,0.73916,1,N
1,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,0.45385,0.54615,1,N
2,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,0.41890,0.58110,1,N
3,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,0.42675,0.57325,1,N
4,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,0.74403,0.25597,0,N
...,...,...,...,...,...
21483374,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,0.65261,0.34739,0,N
21483375,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,0.65261,0.34739,0,N
21483376,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,0.22807,0.77193,1,N
21483377,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,0.40027,0.59973,1,N


In [6]:
df[df['churn'] == 'Y']

,sha2_hash,AGE_GRP10,AGMT_END_SEG,BUNDLE_YN,CH_HH_AVG_MONTH1,CH_LAST_DAYS_BF_GRP,CONTENT_USE_YN,INHOME_RATE,MEDIA_NM_GRP,MONTHS_REMAINING,SCRB_PATH_NM_GRP,STB_RES_1M_YN,SVC_USE_DAYS_GRP,TOTAL_USED_DAYS,TV_I_CNT,VOC_STOP_CANCEL_MONTH1_YN,VOC_TOTAL_MONTH1_YN,p_mt,churn,probability_0,probability_1,prediction
660,0001e7fbf7b042d5859da08a8168b8c355e0b572c16134263d82168f6598b5dc,60대,약정만료후 12개월이상,N,0.00000,3개월내없음,N,0.0,HD,-40,O/B,Y,36개월 이상,3973,2.00000,Y,Y,11,Y,0.05527,0.94473,1
667,0001f92fdce062b1f997938e0efd3c2fcb760b1d7457aac1d23e8ed4e1e25f54,30대,약정만료후 6~9개월,Y,5.62000,3주일전,N,30.0,HD,-5,현장경로,N,36개월 이상,1277,3.00000,Y,Y,8,Y,0.04280,0.95720,1
1270,00039a3592a2093196a591a7919e585df168aa3457f51e3fdbb1e2287043b587,60대,약정만료후 12개월이상,N,0.00000,3개월내없음,N,0.0,HD,-41,O/B,Y,36개월 이상,2351,2.00000,N,N,10,Y,0.37924,0.62076,1
1448,00043eef94e14163de52a1da78d25a33136cd2be189b8a1d595b9c98f548bee1,70대,약정만료후 12개월이상,Y,2.46000,일주일내,N,10.0,HD,-98,O/B,N,36개월 이상,4077,2.00000,N,N,7,Y,0.64349,0.35651,0
1449,00045178e82902c7ea810c83925de83ece3abf2a4f2c2a9a87ca1f5dd0d085ab,60대,약정만료후 12개월이상,N,6.00000,일주일내,N,30.0,HD,-26,I/B,N,36개월 이상,2105,2.00000,N,N,2,Y,0.69294,0.30706,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21482420,fffd11765d5ab229f79b152857181e9f35497d6e7f0225d5dc9927466a7534fa,30대,약정만료전 12개월이상,Y,0.00000,3개월내없음,N,0.0,HD,20,O/B,Y,12개월~24개월미만,501,2.00000,N,N,2,Y,0.33131,0.66869,1
21482870,fffe69a15fe995659e8be4de8c8b6dc72910ee05c7ab5e0a5bc7aeafb7ce8268,80대,약정만료후 12개월이상,N,3.55000,일주일내,N,30.0,HD,-22,O/B,N,36개월 이상,2948,2.00000,N,N,11,Y,0.77641,0.22359,0
21482906,fffea35fc5d88b89a8031befccb61a4f34b03b6d8e5e1f2e8f18747a73eab8db,50대,약정만료전 12개월이상,Y,3.49000,일주일내,Y,0.0,UHD,36,현장경로,Y,6개월미만,11,2.00000,N,Y,5,Y,0.44611,0.55389,1
21483095,ffff2f4cd93de1400bf12b1c97680dd6e87eeb64b7effd67a9a9afbf43c3c103,50대,약정만료전 12개월이상,N,0.62000,일주일전,N,0.0,HD,29,현장경로,N,6개월~12개월미만,231,1.00000,N,N,3,Y,0.56811,0.43189,0


probability_1 = 모델이 생각한 해지율 <br><br>
prediction = 모델이 생각한 해지 여부 (0.5 (threshold) 이상이면 해지라고 예측)

In [8]:
# 일단 유지확률은 지워놓자

df.drop(columns=['probability_0'], axis=1, inplace=True)

In [9]:
df

,sha2_hash,AGE_GRP10,AGMT_END_SEG,BUNDLE_YN,CH_HH_AVG_MONTH1,CH_LAST_DAYS_BF_GRP,CONTENT_USE_YN,INHOME_RATE,MEDIA_NM_GRP,MONTHS_REMAINING,SCRB_PATH_NM_GRP,STB_RES_1M_YN,SVC_USE_DAYS_GRP,TOTAL_USED_DAYS,TV_I_CNT,VOC_STOP_CANCEL_MONTH1_YN,VOC_TOTAL_MONTH1_YN,p_mt,churn,probability_1,prediction
0,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,60대,약정만료전 12개월이상,Y,0.00000,3개월내없음,N,10.0,HD,13,I/B,N,12개월~24개월미만,733,3.00000,N,N,2,N,0.73916,1
1,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,60대,약정만료전 9~12개월,Y,6.72000,일주일내,N,0.0,HD,12,I/B,Y,24개월~ 36개월미만,764,3.00000,N,Y,3,N,0.54615,1
2,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,60대,약정만료전 9~12개월,Y,9.86000,3주일전,N,10.0,HD,11,I/B,N,24개월~ 36개월미만,794,3.00000,N,N,4,N,0.58110,1
3,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,60대,약정만료전 9~12개월,Y,5.95000,4주일전,N,10.0,HD,10,I/B,N,24개월~ 36개월미만,825,3.00000,N,N,5,N,0.57325,1
4,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,60대,약정만료전 6~9개월,Y,4.03000,일주일내,N,0.0,HD,9,I/B,N,24개월~ 36개월미만,855,3.00000,N,N,6,N,0.25597,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21483374,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,50대,약정만료후 12개월이상,N,2.03000,일주일내,N,10.0,HD,-40,현장경로,N,36개월 이상,2338,1.00000,N,N,8,N,0.34739,0
21483375,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,50대,약정만료후 12개월이상,N,2.91000,일주일내,N,10.0,HD,-41,현장경로,N,36개월 이상,2368,1.00000,N,N,9,N,0.34739,0
21483376,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,50대,약정만료후 12개월이상,N,1.97000,2주일전,N,10.0,HD,-42,현장경로,N,36개월 이상,2399,1.00000,N,Y,10,N,0.77193,1
21483377,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,50대,약정만료후 12개월이상,N,0.00000,3개월내없음,N,0.0,HD,-43,현장경로,N,36개월 이상,2429,1.00000,N,N,11,N,0.59973,1


### 모델이 1(해지)로 예측했을 때, 실제 해지인 사람들 위주로 확인해보자.
실제 해지인데, 모델이 1로 예측할만큼 가장 위험한 사람들이다.

In [10]:
check_df = df[(df['churn'] == 'Y') & (df['prediction'] == 1)]

In [11]:
check_df['probability_1'].describe()

count   95683.00000
mean        0.79046
std         0.14357
min         0.50001
25%         0.66188
50%         0.82038
75%         0.92286
max         0.96652
Name: probability_1, dtype: float64

월 별로 확인해보자

In [12]:
# 2월
print("2월")
display(check_df[(check_df['p_mt'] == 2)]['probability_1'].describe())
print("--------------------------------------")

# 3월
print("3월")
display(check_df[(check_df['p_mt'] == 3)]['probability_1'].describe())
print("--------------------------------------")

# 4월
print("4월")
display(check_df[(check_df['p_mt'] == 4)]['probability_1'].describe())
print("--------------------------------------")

# 5월
print("5월")
display(check_df[(check_df['p_mt'] == 5)]['probability_1'].describe())
print("--------------------------------------")

# 6월
print("6월")
display(check_df[(check_df['p_mt'] == 6)]['probability_1'].describe())
print("--------------------------------------")

# 7월
print("7월")
display(check_df[(check_df['p_mt'] == 7)]['probability_1'].describe())
print("--------------------------------------")

# 8월
print("8월")
display(check_df[(check_df['p_mt'] == 8)]['probability_1'].describe())
print("--------------------------------------")

# 9월
print("9월")
display(check_df[(check_df['p_mt'] == 9)]['probability_1'].describe())
print("--------------------------------------")

# 10월
print("10월")
display(check_df[(check_df['p_mt'] == 10)]['probability_1'].describe())
print("--------------------------------------")

# 11월
print("11월")
display(check_df[(check_df['p_mt'] == 11)]['probability_1'].describe())
print("--------------------------------------")

2월


count   9718.00000
mean       0.77654
std        0.14446
min        0.50006
25%        0.64346
50%        0.79718
75%        0.91396
max        0.96648
Name: probability_1, dtype: float64

--------------------------------------
3월


count   9205.00000
mean       0.78745
std        0.14481
min        0.50008
25%        0.65669
50%        0.81517
75%        0.92173
max        0.96648
Name: probability_1, dtype: float64

--------------------------------------
4월


...

--------------------------------------
10월


count   10474.00000
mean        0.79455
std         0.14257
min         0.50009
25%         0.66724
50%         0.82815
75%         0.92459
max         0.96648
Name: probability_1, dtype: float64

--------------------------------------
11월


count   9762.00000
mean       0.77976
std        0.14233
min        0.50012
25%        0.65138
50%        0.80215
75%        0.91417
max        0.96648
Name: probability_1, dtype: float64

--------------------------------------


check_df의 2월 평균 해지율 : 0.77654<br><br>
check_df의 3월 평균 해지율 : 0.78745<br><br>
check_df의 4월 평균 해지율 : 0.78467<br><br>
check_df의 5월 평균 해지율 : 0.79651<br><br>
check_df의 6월 평균 해지율 : 0.79425<br><br>
check_df의 7월 평균 해지율 : 0.79330<br><br>
check_df의 8월 평균 해지율 : 0.80177<br><br>
check_df의 9월 평균 해지율 : 0.79566<br><br>
check_df의 10월 평균 해지율 : 0.79455<br><br>
check_df의 11월 평균 해지율 : 0.77976<br><br>

실제 해지인데, 모델이 1로 예측할만큼 가장 위험한 사람들의 평균 해지율은 0.8 정도이다. <br><br>
그러므로, **0.8 이상**은 "매우 위험"에 속하는 고객들로 간주한다.

---

### 모델이 0(유지)으로 예측했을 때, 실제 유지인 사람들 위주로 확인해보자.
실제 유지인데, 모델이 0으로 예측한만큼 가장 안전한 사람들이다.

In [13]:
check_df2 = df[(df['churn'] == 'N') & (df['prediction'] == 0)]

In [14]:
check_df2['probability_1'].describe()

count   17177768.00000
mean           0.26783
std            0.10006
min            0.10570
25%            0.18858
50%            0.25707
75%            0.33946
max            0.50000
Name: probability_1, dtype: float64

월 별로 확인

In [15]:
# 2월
print("2월")
display(check_df2[(check_df2['p_mt'] == 2)]['probability_1'].describe())
print("--------------------------------------")

# 3월
print("3월")
display(check_df2[(check_df2['p_mt'] == 3)]['probability_1'].describe())
print("--------------------------------------")

# 4월
print("4월")
display(check_df2[(check_df2['p_mt'] == 4)]['probability_1'].describe())
print("--------------------------------------")

# 5월
print("5월")
display(check_df2[(check_df2['p_mt'] == 5)]['probability_1'].describe())
print("--------------------------------------")

# 6월
print("6월")
display(check_df2[(check_df2['p_mt'] == 6)]['probability_1'].describe())
print("--------------------------------------")

# 7월
print("7월")
display(check_df2[(check_df2['p_mt'] == 7)]['probability_1'].describe())
print("--------------------------------------")

# 8월
print("8월")
display(check_df2[(check_df2['p_mt'] == 8)]['probability_1'].describe())
print("--------------------------------------")

# 9월
print("9월")
display(check_df2[(check_df2['p_mt'] == 9)]['probability_1'].describe())
print("--------------------------------------")

# 10월
print("10월")
display(check_df2[(check_df2['p_mt'] == 10)]['probability_1'].describe())
print("--------------------------------------")

# 11월
print("11월")
display(check_df2[(check_df2['p_mt'] == 11)]['probability_1'].describe())
print("--------------------------------------")

2월


count   1552672.00000
mean          0.26810
std           0.10019
min           0.10570
25%           0.18912
50%           0.25657
75%           0.33909
max           0.50000
Name: probability_1, dtype: float64

--------------------------------------
3월


count   1549194.00000
mean          0.27357
std           0.10020
min           0.10570
25%           0.19643
50%           0.26560
75%           0.34536
max           0.50000
Name: probability_1, dtype: float64

--------------------------------------
4월


...

--------------------------------------
10월


count   1556088.00000
mean          0.26600
std           0.09964
min           0.10570
25%           0.18566
50%           0.25669
75%           0.33658
max           0.50000
Name: probability_1, dtype: float64

--------------------------------------
11월


count   1557375.00000
mean          0.26861
std           0.10079
min           0.10570
25%           0.19007
50%           0.25394
75%           0.34110
max           0.50000
Name: probability_1, dtype: float64

--------------------------------------


check_df의 2월 평균 해지율 : 0.26810<br><br>
check_df의 3월 평균 해지율 : 0.27357<br><br>
check_df의 4월 평균 해지율 : 0.26757<br><br>
check_df의 5월 평균 해지율 : 0.26879<br><br>
check_df의 6월 평균 해지율 : 0.26906<br><br>
check_df의 7월 평균 해지율 : 0.26805<br><br>
check_df의 8월 평균 해지율 : 0.26785<br><br>
check_df의 9월 평균 해지율 : 0.26306<br><br>
check_df의 10월 평균 해지율 : 0.26600<br><br>
check_df의 11월 평균 해지율 : 0.26861<br><br>

실제 유지인데, 모델이 0으로 예측할만큼 가장 위험한 사람들의 평균 해지율은 0.26 정도이다. <br><br>
그러므로, **0.25 이하**는 "안정"에 속하는 고객들로 간주한다.

가운데 구간을 어떻게 나눌지 몰라서, <br><br>
일단 가운데 구간을 3개의 구간으로 나눈다.<br><br>
1. 0.8 이상 (매우 위험)
2. **0.8 미만 0.6 이상**
3. **0.6 미만 0.4 이상**
4. **0.4 미만 0.25 초과**
5. 0.25 이하 (안정)

In [16]:
# 구간 나누기
def classify_customer_fine(probability):
    if probability >= 0.8:
        return '매우 위험'
    elif probability >= 0.6:
        return '위험'
    elif probability >= 0.4:
        return '주의'
    elif probability > 0.25:
        return '양호'
    else:
        return '안정'
# 분류 적용
df['customer_category'] = df['probability_1'].apply(classify_customer_fine)

In [17]:
df

,sha2_hash,AGE_GRP10,AGMT_END_SEG,BUNDLE_YN,CH_HH_AVG_MONTH1,CH_LAST_DAYS_BF_GRP,CONTENT_USE_YN,INHOME_RATE,MEDIA_NM_GRP,MONTHS_REMAINING,SCRB_PATH_NM_GRP,STB_RES_1M_YN,SVC_USE_DAYS_GRP,TOTAL_USED_DAYS,TV_I_CNT,VOC_STOP_CANCEL_MONTH1_YN,VOC_TOTAL_MONTH1_YN,p_mt,churn,probability_1,prediction,customer_category
0,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,60대,약정만료전 12개월이상,Y,0.00000,3개월내없음,N,10.0,HD,13,I/B,N,12개월~24개월미만,733,3.00000,N,N,2,N,0.73916,1,위험
1,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,60대,약정만료전 9~12개월,Y,6.72000,일주일내,N,0.0,HD,12,I/B,Y,24개월~ 36개월미만,764,3.00000,N,Y,3,N,0.54615,1,주의
2,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,60대,약정만료전 9~12개월,Y,9.86000,3주일전,N,10.0,HD,11,I/B,N,24개월~ 36개월미만,794,3.00000,N,N,4,N,0.58110,1,주의
3,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,60대,약정만료전 9~12개월,Y,5.95000,4주일전,N,10.0,HD,10,I/B,N,24개월~ 36개월미만,825,3.00000,N,N,5,N,0.57325,1,주의
4,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,60대,약정만료전 6~9개월,Y,4.03000,일주일내,N,0.0,HD,9,I/B,N,24개월~ 36개월미만,855,3.00000,N,N,6,N,0.25597,0,양호
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21483374,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,50대,약정만료후 12개월이상,N,2.03000,일주일내,N,10.0,HD,-40,현장경로,N,36개월 이상,2338,1.00000,N,N,8,N,0.34739,0,양호
21483375,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,50대,약정만료후 12개월이상,N,2.91000,일주일내,N,10.0,HD,-41,현장경로,N,36개월 이상,2368,1.00000,N,N,9,N,0.34739,0,양호
21483376,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,50대,약정만료후 12개월이상,N,1.97000,2주일전,N,10.0,HD,-42,현장경로,N,36개월 이상,2399,1.00000,N,Y,10,N,0.77193,1,위험
21483377,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,50대,약정만료후 12개월이상,N,0.00000,3개월내없음,N,0.0,HD,-43,현장경로,N,36개월 이상,2429,1.00000,N,N,11,N,0.59973,1,주의


In [18]:
# 고객 카테고리별 개수 계산
category_counts = df['customer_category'].value_counts().reset_index()
category_counts.columns = ['고객 위험도 분류', '고객 수']

In [19]:
# 천 단위 쉼표 추가
category_counts['고객 수'] = category_counts['고객 수'].apply(lambda x: f"{x:,}")

In [20]:
# 특정 순서로 정렬
custom_order = ['매우 위험', '위험', '주의', '양호', '안정']
category_counts = category_counts.set_index('고객 위험도 분류').reindex(custom_order).reset_index()

In [21]:
display(category_counts)

,고객 위험도 분류,고객 수
0,매우 위험,"501,040"
1,위험,"1,896,913"
2,주의,"4,158,202"
3,양호,"6,717,283"
4,안정,"8,209,941"


In [22]:
# 결과 확인
display(df['customer_category'].value_counts())

customer_category
안정       8209941
양호       6717283
주의       4158202
위험       1896913
매우 위험     501040
Name: count, dtype: int64

**안정 > 양호 > 주의 > 위험 > 매우 위험** 순으로 고객이 배치되어있다. <br><br>
위험도가 낮을수록 고객의 수가 점진적으로 낮아짐. 자연스럽게 계층적으로 구분이 되어 있음.
- 안정 : 약 39%
- 양호 : 약 31%
- 주의 : 약 19%
- 위험 : 약 9%
- 매우 위험 : 약 2%

### ⭐ 이제, 각 Segmentation별 실제 해지율을 확인해보자.

In [23]:
# 그룹별 평균 확률과 해지율 계산
df['churn'] = df['churn'].map({'Y': 1, 'N': 0})

group_stats = df.groupby('customer_category').agg({
    # 해당 그룹의 실제 해지율을 보여줍니다.
    'churn': 'mean'
})

In [24]:
group_stats = group_stats.sort_values(by='churn', ascending=False)

In [25]:
group_stats = group_stats.sort_values(by='churn', ascending=False)
print("")
display(group_stats)

,churn
customer_category,
매우 위험,0.10338
위험,0.01566
주의,0.00604
양호,0.00265
안정,0.00107


위험도가 높은 그룹일수록 해지율이 증가하는 완벽한 계층적 구조를 보임. <br><br>
특정 그룹에 데이터가 과도하게 몰려있지 않고, **"안정 → 양호 → 주의 → 위험 → 매우 위험"** 순으로 자연스럽게 계층화 되어있음. <br><br>
"매우 위험" 고객의 해지율이 10.34%, "위험" 고객이 1.57%, "안정" 고객이 0.11%로, **위험도가 높을수록 해지율이 높게 나타남.** <br><br>
"매우 위험" 고객은 "안정" 고객 대비 해지율이 **약 94배 높음**!!<br><br>
이는 모델이 해지 가능성이 높은 고객을 "위험군"으로 정확히 분류하고 있다는 증거!

### ✅ 결론 : 고객의 구간을 총 5개의 구간으로 나누고, 매우 위험, 위험, 주의 구간이 위험구간으로 파악됨.

파일 저장

In [26]:
df['churn'] = df['churn'].map({1:'Y', 0:'N'})

In [28]:
df.rename(columns={'probability_1':'churn_rate'}, inplace=True)

In [30]:
display(df)

,sha2_hash,AGE_GRP10,AGMT_END_SEG,BUNDLE_YN,CH_HH_AVG_MONTH1,CH_LAST_DAYS_BF_GRP,CONTENT_USE_YN,INHOME_RATE,MEDIA_NM_GRP,MONTHS_REMAINING,SCRB_PATH_NM_GRP,STB_RES_1M_YN,SVC_USE_DAYS_GRP,TOTAL_USED_DAYS,TV_I_CNT,VOC_STOP_CANCEL_MONTH1_YN,VOC_TOTAL_MONTH1_YN,p_mt,churn,churn_rate,prediction,customer_category
0,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,60대,약정만료전 12개월이상,Y,0.00000,3개월내없음,N,10.0,HD,13,I/B,N,12개월~24개월미만,733,3.00000,N,N,2,N,0.73916,1,위험
1,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,60대,약정만료전 9~12개월,Y,6.72000,일주일내,N,0.0,HD,12,I/B,Y,24개월~ 36개월미만,764,3.00000,N,Y,3,N,0.54615,1,주의
2,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,60대,약정만료전 9~12개월,Y,9.86000,3주일전,N,10.0,HD,11,I/B,N,24개월~ 36개월미만,794,3.00000,N,N,4,N,0.58110,1,주의
3,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,60대,약정만료전 9~12개월,Y,5.95000,4주일전,N,10.0,HD,10,I/B,N,24개월~ 36개월미만,825,3.00000,N,N,5,N,0.57325,1,주의
4,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,60대,약정만료전 6~9개월,Y,4.03000,일주일내,N,0.0,HD,9,I/B,N,24개월~ 36개월미만,855,3.00000,N,N,6,N,0.25597,0,양호
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21483374,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,50대,약정만료후 12개월이상,N,2.03000,일주일내,N,10.0,HD,-40,현장경로,N,36개월 이상,2338,1.00000,N,N,8,N,0.34739,0,양호
21483375,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,50대,약정만료후 12개월이상,N,2.91000,일주일내,N,10.0,HD,-41,현장경로,N,36개월 이상,2368,1.00000,N,N,9,N,0.34739,0,양호
21483376,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,50대,약정만료후 12개월이상,N,1.97000,2주일전,N,10.0,HD,-42,현장경로,N,36개월 이상,2399,1.00000,N,Y,10,N,0.77193,1,위험
21483377,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,50대,약정만료후 12개월이상,N,0.00000,3개월내없음,N,0.0,HD,-43,현장경로,N,36개월 이상,2429,1.00000,N,N,11,N,0.59973,1,주의


In [18]:
df.to_csv("data/TPS_cancel_data_CustomerSeg.csv", index=False)